In [ ]:
# from google.colab import drive
# import os,cv2, sys, math , csv
# drive.mount('/content/drive')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # showing and rendering figures
# io related
from skimage.io import imread
import os
from glob import glob
# not needed in Kaggle, but required in Jupyter
%matplotlib inline 
import albumentations as albu

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf



from tensorflow.keras.applications import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.utils import *

from sklearn.metrics import *
from sklearn.model_selection import *
import tensorflow.keras.backend as K


import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from skimage.io import *

In [ ]:
base_image_dir = "../input/eye-dataset/Eye_PACS/input/diabetic-retinopathy-detection"
retina_df = pd.read_csv(os.path.join(base_image_dir, 'Train_labels.csv'))
retina_df['PatientId'] = retina_df['image'].map(lambda x: x.split('_')[0])
retina_df['path'] = retina_df['image'].map(lambda x: os.path.join(base_image_dir,'train_512/'
                                                         '{}.jpeg'.format(x)))
retina_df['exists'] = retina_df['path'].map(os.path.exists)
print(retina_df['exists'].sum(), 'images found of', retina_df.shape[0], 'total')
retina_df['eye'] = retina_df['image'].map(lambda x: 1 if x.split('_')[-1]=='left' else 0)
from keras.utils.np_utils import to_categorical
retina_df['level_cat'] = retina_df['level'].map(lambda x: to_categorical(x, 1+retina_df['level'].max()))

retina_df.dropna(inplace = True)
retina_df = retina_df[retina_df['exists']]
retina_df.sample(10)

In [ ]:
retina_df[['level', 'eye']].hist(figsize = (10, 5))

In [ ]:
from sklearn.model_selection import train_test_split
rr_df = retina_df[['PatientId', 'level']].drop_duplicates()
train_ids, valid_ids = train_test_split(rr_df['PatientId'], 
                                   test_size = 0.25, 
                                   random_state = 2018,
                                   stratify = rr_df['level'])
raw_train_df = retina_df[retina_df['PatientId'].isin(train_ids)]
valid_df = retina_df[retina_df['PatientId'].isin(valid_ids)]
rr_df = valid_df[['PatientId', 'level']].drop_duplicates()
valid_ids, test_ids = train_test_split(rr_df['PatientId'], 
                                   test_size = 0.5, 
                                   random_state = 2018,
                                   stratify = rr_df['level'])
valid_df = retina_df[retina_df['PatientId'].isin(valid_ids)].reset_index(drop = True)
test_df = retina_df[retina_df['PatientId'].isin(test_ids)].reset_index(drop = True)
print('train', raw_train_df.shape[0], 'validation', valid_df.shape[0], 'test', test_df.shape[0])


In [ ]:
train_df = raw_train_df.groupby(['level', 'eye']).apply(lambda x: x.sample(750, replace = True)
                                                      ).reset_index(drop = True)
print('New Data Size:', train_df.shape[0], 'Old Size:', raw_train_df.shape[0])
train_df[['level', 'eye']].hist(figsize = (10, 5))

In [ ]:
print(valid_df['path'].iloc[1])

In [ ]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    
    def __init__(self, df,  batch_size=16, dim=(224,224,3), shuffle=True, transform= None):
        'Initialization'
        self.dim = dim
        self.df = df
        self.transform=transform
        #self.class_map = class_map
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [k for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
        print("success")

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        batch_imgs = list()
        batch_labels = list()

        # Generate data
        for i in list_IDs_temp:
            # Store sample
            img = load_img(self.df['path'][i] ,target_size=self.dim)
            #img= (img, (224,224), interpolation=cv2.INTER_LINEAR)
            
            img = img_to_array(img)/255.
            if(self.transform != None):
                augmented = self.transform(image=img)
                img = augmented["image"]
#                 mask = augmented["mask"]
            batch_imgs.append(img)

            label = self.df['level_cat'][i]
            batch_labels.append(label)

        return np.array(batch_imgs) ,np.array(batch_labels)

In [ ]:
print(train_df)

In [ ]:
import cv2
transform = albu.Compose([
    albu.HorizontalFlip(p=0.5),
    albu.VerticalFlip(p=0.5),
    albu.CropAndPad (percent = -0.2, keep_size=True, interpolation=cv2.INTER_NEAREST, p=0.5),
    albu.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit= 15, interpolation= cv2.INTER_NEAREST,
                                          border_mode= cv2.BORDER_REPLICATE, p=0.5)
])

In [ ]:
img_size = 224
#class_map = class_palette()

train_generator1 = DataGenerator(train_df,batch_size=4, dim=(img_size,img_size,3) ,shuffle=True, transform=transform)
X,y = train_generator1.__getitem__(0)
test_generator1 = DataGenerator(test_df,batch_size=4, dim=(img_size,img_size,3) ,shuffle=False)
X_test,y_test = test_generator1.__getitem__(0)
print(X.shape, y.shape)
print(X_test.shape, y_test.shape)

In [ ]:
val_generator = DataGenerator(valid_df,  batch_size=4, dim=(img_size,img_size,3) ,shuffle=False)

In [ ]:
print(len(X),len(y))
print(val_generator)

# R

In [ ]:
# Initializing a Sequential model
model = Sequential()
model.add(VGG19(input_shape=(224,224,3),include_top=False,weights="imagenet"))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
# Creating an output layer
model.add(Dense(units= 5, activation='softmax'))

for layers in model.layers[:]:
    layers.trainable = True

c3=tf.keras.callbacks.ReduceLROnPlateau(
    monitor="loss",
    factor=0.1,
    patience=2,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0.001
)

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['categorical_accuracy','AUC'])
#load_model("/content/drive/MyDrive/Workshop2022/diabetes_retinopathy.h5")
history=model.fit(train_generator1,epochs= 10,callbacks=[c3], validation_data= val_generator )

In [ ]:
model.save("/content/drive/MyDrive/Workshop2022/diabetes_retinopathy.h5")

In [ ]:

x_test=[]
y_test =[]
for i in test_df['path']:
    x = cv2.cvtColor(cv2.imread(i), cv2.COLOR_BGR2RGB)
    
    x= cv2.resize(x, (224,224))
    x_test.append(x)
for i in test_df['level_cat']:
    y_test.append(i)
x_test = np.array(x_test)
y_test = np.array(y_test)
print(x_test.shape)
print(y_test.shape)

In [ ]:
Y_pred=[]
for i in range (len(x_test)):
    y_pred = model.predict(np.expand_dims(x_test[i], axis=0))
    Y_pred.append(y_pred)
Y_pred= np.asarray(Y_pred)

In [ ]:
print(Y_pred.shape)

In [ ]:
Y_pred= np.squeeze(Y_pred, axis=1)
print(Y_pred.shape)

In [ ]:
Y_test=np.argmax(y_test, axis=1)
Y_pred=np.argmax(Y_pred, axis=1)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


print(Y_pred.shape , Y_test.shape)
#pred=np.argmax(model.predict(x_test),axis=1)
cm=confusion_matrix(Y_test,Y_pred)
cm_plot=ConfusionMatrixDisplay(cm,display_labels=['0','1','2','3','4'])
cm_plot.plot()
plt.show()

In [ ]:
print("Performance Report:")
# y_pred6=np.argmax(model.predict(x_test),axis=-1)

# y_pred_prb6=model.predict(x_test)
target=['0','1','2','3','4']
from sklearn import metrics
print('Accuracy score is :', metrics.accuracy_score(Y_test, Y_pred))
print('Precision score is :', metrics.precision_score(Y_test, Y_pred, average='weighted'))
print('Recall score is :',metrics.recall_score(Y_test,Y_pred, average='weighted'))
print('F1 Score is :', metrics.f1_score(Y_test, Y_pred,average='weighted'))
print('Cohen Kappa Score:', metrics.cohen_kappa_score(Y_test,Y_pred))
#print('\t\tClassification Report:\n', metrics.classification_report(Y_test,pred,target_names=target))

In [ ]:
plt.title("Test Image")
plt.imshow(x_test[1])
plt.show()
x_test1 = x_test[1]
x_test1 = x_test1[np.newaxis,...]
y_test3=np.argmax(y_test[1], axis=-1)
pred=np.argmax(model.predict(x_test1),axis=-1)
print("Level=", pred)

In [ ]:
# np.argmax(y_val,axis=-1)

In [ ]:
model.summary()
